In [1]:
import sys
sys.executable

'/opt/conda/bin/python'

In [2]:
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d import proj3d
from skimage.transform import resize
from scipy.spatial import distance
import pandas as pd
from tqdm import tqdm
from imageio import imread
import glob

ModuleNotFoundError: No module named 'sklearn'

In [3]:
train_df = pd.read_csv("./data/train_relationships.csv")
test_df = pd.read_csv("./data/sample_submission.csv")

In [4]:
import sys
import pickle
sys.path.append('/home/macul/Documents/macul/mklib/utils/')
from mxFaceFeatureExtract import mxFaceFeatureExtract
extractor=mxFaceFeatureExtract('/home/macul/Documents/macul/FaceRecog_ResNet_MX/output/server_train18','train_18',62, outputs_name={'embedding':'embedding_output'},mean_value='/home/macul/Documents/macul/FaceRecog_ResNet_MX/dataset_insight/mean.json')

/home/macul/Documents/macul/FaceRecog_ResNet_MX/output/server_train18/train_18-0062.params


In [5]:
def calc_embs(filepaths):
    pd = []
    for f in filepaths:
        #print(extractor.getEmbedding(f))
        pd.append([extractor.getEmbedding(f)])

    return np.concatenate(pd)

In [6]:
test_images = os.listdir("./data/test/")
test_embs = calc_embs([os.path.join("./data/test/", f) for f in test_images])
np.save("test_embs.npy", test_embs)

In [7]:
test_df["distance"] = 0
img2idx = dict()
for idx, img in enumerate(test_images):
    img2idx[img] = idx

In [8]:
for idx, row in tqdm(test_df.iterrows(), total=len(test_df)):
    imgs = [test_embs[img2idx[img]] for img in row.img_pair.split("-")]
    #test_df.loc[idx, "distance"] = 1.0-np.dot(*imgs)
    #test_df.loc[idx, "distance"] = distance.euclidean(*imgs)
    test_df.loc[idx, "distance"] = np.dot(*imgs)

100%|██████████| 5310/5310 [00:02<00:00, 2315.36it/s]


In [9]:
all_distances = test_df.distance.values
sum_dist = np.sum(all_distances)

In [10]:
np.max(all_distances)

0.8950536251068115

In [17]:
probs = []
for dist in tqdm(all_distances):
    #prob = np.sum(all_distances[np.where(all_distances <= dist)[0]])/sum_dist
    #probs.append(1 - prob)
    probs.append(int(dist>-0.1))

100%|██████████| 5310/5310 [00:00<00:00, 329868.84it/s]


In [18]:
sub_df = pd.read_csv("./data/sample_submission.csv")
sub_df.is_related = probs
sub_df.to_csv("submission.csv", index=False)

In [27]:
test_embs[img2idx['face02240.jpg']]

-0.026886275

In [10]:
embedding=extractor.getEmbedding('./data/test/face02240.jpg')

In [11]:
embedding1=extractor.getEmbedding('./data/test/face02336.jpg')

In [12]:
np.linalg.norm(embedding)

1.0

In [13]:
np.dot(embedding,embedding1)

0.46142876

In [50]:
distance.euclidean(embedding,embedding1)

1.353723168373108

In [27]:
from PIL import Image

In [29]:
im1=Image.open('./data/test/face02240.jpg')
im1.show()